In [2]:
import numpy as np
import pandas as pd
import json
import re
#must pip install textstat
# import textstat
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
import gensim
# nltk.download('punkt')

C:\Users\Risha\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Begin by reading in reddit json data

In [3]:
# with open('/Users/jayshenoy/Documents/DataScienceLabs/Final_Project/data_checkpoint.csv') as json_file:      
#     data = json_file.readlines()
#     # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
#     data = list(map(json.loads, data)) 
# frame=pd.DataFrame(data)
# frame.head()
frame=pd.read_csv('engineered_reddit_train.csv', lineterminator='\n')
frame.head()

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,num_user_mentions,num_subreddit_mentions,has_flair,num_outside_links,num_reddit_links,title_is_question,num_questions,readability,uppercase_ratio,length
0,YouthfulPhotographer,NaN,NaN,Welcome to generation void,True,0.0,1517443200,False,False,0,...,0,0,False,0,0,0,0,13.1,0.037037,27
1,jasonklacour,NaN,NaN,Welcome,True,0.0,1517443200,False,False,0,...,0,0,False,0,0,0,0,8.4,0.142857,7
2,Assassin2000,NaN,NaN,I'm 16 and the friend told me he was joking af...,True,0.0,1517443200,False,False,0,...,0,0,False,0,0,0,0,6.4,0.031250,64
3,kawaiicicle,Employee,Assistant Manager,What? \r It’s a niche rpg. Most rpg fans are a...,True,0.0,1517443200,False,False,0,...,0,0,True,0,0,1,2,2.9,0.044643,112
4,recklessmaterialism,NaN,NaN,solid!,True,0.0,1517443200,False,False,0,...,0,0,False,0,0,0,0,8.4,0.000000,6


In [7]:
len(frame)

2885987

In [ ]:
def clean_bodies_df(df):
    df['body'] = pd.Series([body.replace("\n", " ") for body in df['body']])
    return df

cleanedframe=clean_bodies_df(frame)

In [4]:
cleanedframe=frame
print(cleanedframe['body'].isna().sum())
print(cleanedframe['body'].isnull().sum())

0
0


In [5]:
#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(cleanedframe['body'])]
tagged_data=[]
c=0
for i, _d in enumerate(cleanedframe['body']):
    tagged_data.append(TaggedDocument(words=word_tokenize(_d.lower()),tags=[str(i)]))
    c+=1
    if c%25000==0:
        print(c)

25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000
475000
500000
525000
550000
575000
600000
625000
650000
675000
700000
725000
750000
775000
800000
825000
850000
875000
900000
925000
950000
975000
1000000
1025000
1050000
1075000
1100000
1125000
1150000
1175000
1200000
1225000
1250000
1275000
1300000
1325000
1350000
1375000
1400000
1425000
1450000
1475000
1500000
1525000
1550000
1575000
1600000
1625000
1650000
1675000
1700000
1725000
1750000
1775000
1800000
1825000
1850000
1875000
1900000
1925000
1950000
1975000
2000000
2025000
2050000
2075000
2100000
2125000
2150000
2175000
2200000
2225000
2250000
2275000
2300000
2325000
2350000
2375000
2400000
2425000
2450000
2475000
2500000
2525000
2550000
2575000
2600000
2625000
2650000
2675000
2700000
2725000
2750000
2775000
2800000
2825000
2850000
2875000


In [6]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Risha\Documents\Data Science Lab\Final Project


In [16]:
import datetime

In [8]:
from gensim.test.utils import common_corpus, common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

class EpochSaver(CallbackAny2Vec):

    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0

    def on_epoch_end(self, model):
#         output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
#         model.save(output_path)
        model.save('newest_small_d2v_model.model')
        self.epoch += 1



In [9]:
class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        print(datetime.datetime.now())

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        print(datetime.datetime.now())
        self.epoch += 1

In [8]:
from gensim.models.callbacks import PerplexityMetric
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import common_corpus, common_dictionary

# Log the perplexity score at the end of each epoch.
perplexity_logger = PerplexityMetric(corpus=common_corpus, logger='shell')
lda = LdaModel(common_corpus, id2word=common_dictionary, num_topics=5, callbacks=[perplexity_logger])

In [10]:
max_epochs = 300
vec_size = 50
alpha = 0.01
saver=EpochSaver("mdlo")
logger=EpochLogger()
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.0001,
                min_count=1,
                subsaplming=1e-6,
                window_size=5,
                workers=4,
                dm=1,callbacks=[saver,logger])
  
model.build_vocab(tagged_data)

C:\Users\Risha\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [11]:
print(len(tagged_data))

2885987


In [12]:
model.corpus_count

2885987

In [20]:
import gensim

In [14]:
print(gensim.models.doc2vec.FAST_VERSION > -1)
import datetime

True


In [15]:
# model.train(tagged_data, total_examples=model.corpus_count, epochs=500)
# model.save("d2v.model")
# print("Model Saved")
# original trained for ~125 epochs over ~9 hours
for epoch in range(500):
    print("epoch:",epoch)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=1)
    model.alpha -= 0.00002  # decrease the learning rate
    model.min_alpha = model.alpha
model.save("small_d2v.model")
print("Model Saved")

epoch: 0
Epoch #0 start
2018-12-15 23:58:01.261218
Epoch #0 end
2018-12-16 00:01:25.449231
epoch: 1
Epoch #1 start
2018-12-16 00:01:25.450228
Epoch #1 end
2018-12-16 00:04:54.423908
epoch: 2
Epoch #2 start
2018-12-16 00:04:54.423908
Epoch #2 end
2018-12-16 00:08:20.248454
epoch: 3
Epoch #3 start
2018-12-16 00:08:20.248454
Epoch #3 end
2018-12-16 00:11:44.530644
epoch: 4
Epoch #4 start
2018-12-16 00:11:44.530644
Epoch #4 end
2018-12-16 00:15:14.218782
epoch: 5
Epoch #5 start
2018-12-16 00:15:14.218782
Epoch #5 end
2018-12-16 00:18:46.238407
epoch: 6
Epoch #6 start
2018-12-16 00:18:46.238407
Epoch #6 end
2018-12-16 00:22:09.631607
epoch: 7
Epoch #7 start
2018-12-16 00:22:09.631607
Epoch #7 end
2018-12-16 00:25:34.154058
epoch: 8
Epoch #8 start
2018-12-16 00:25:34.154058
Epoch #8 end
2018-12-16 00:28:59.284395
epoch: 9
Epoch #9 start
2018-12-16 00:28:59.284395
Epoch #9 end
2018-12-16 00:32:23.531510
epoch: 10
Epoch #10 start
2018-12-16 00:32:23.532506
Epoch #10 end
2018-12-16 00:35:45.957

KeyboardInterrupt: 

In [ ]:
docu

In [5]:
frame.describe()

,controversiality,created_utc,gilded,retrieved_on,score
count,3.194211e+06,3.194211e+06,3.194211e+06,3.194211e+06,3.194211e+06
mean,2.043509e-02,1.517490e+09,1.984841e-04,1.518807e+09,8.892290e+00
std,1.414832e-01,2.646685e+04,1.729891e-02,7.827520e+04,1.245221e+02
min,0.000000e+00,1.517443e+09,0.000000e+00,1.518478e+09,-8.660000e+02
25%,0.000000e+00,1.517464e+09,0.000000e+00,1.518816e+09,1.000000e+00
50%,0.000000e+00,1.517497e+09,0.000000e+00,1.518825e+09,2.000000e+00
75%,0.000000e+00,1.517513e+09,0.000000e+00,1.518833e+09,4.000000e+00
max,1.000000e+00,1.517530e+09,1.200000e+01,1.518842e+09,4.832700e+04


author_cakeday,id, link_id,retrieved_on,subreddit_id

In [6]:
#frame=frame.drop(columns=['author_cakeday','id','link_id','retrieved_on','subreddit_id'])
frame=frame.drop(columns='parent_id')

In [36]:
frame.to_csv("clean_reddit_02_01_18.csv",index=False)

In [38]:
df=pd.read_csv("clean_reddit_02_01_18.csv")
df.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5,6,9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,is_submitter,permalink,score,stickied,subreddit,subreddit_type
0,YouthfulPhotographer,NaN,NaN,Welcome to generation void,True,0,1.51744e+09,NaN,False,0,False,/r/tumblr/comments/7uaobc/nihilism_across_gene...,7,False,tumblr,public
1,jasonklacour,NaN,NaN,Welcome,True,0,1.51744e+09,NaN,False,0,False,/r/pics/comments/7ude45/7_years_later_im_offic...,1,False,pics,public
2,Assassin2000,NaN,NaN,I'm 16 and the friend told me he was joking af...,True,0,1.51744e+09,NaN,False,0,True,/r/legaladvice/comments/7uegdc/took_a_awful_jo...,1,False,legaladvice,public
3,kawaiicicle,Employee,Assistant Manager,What? \nIt’s a niche rpg. Most rpg fans are ad...,True,0,1.51744e+09,NaN,False,0,False,/r/GameStop/comments/7u7mps/is_this_possible/d...,0,False,GameStop,public
4,recklessmaterialism,NaN,NaN,solid!,True,0,1.51744e+09,NaN,False,0,True,/r/Seattle/comments/7udrqf/in_town_for_only_a_...,-1,False,Seattle,public


In [24]:
#generating flesch_kincaid_readability
#vocabulary difficulty
def generatefkReadability(frame):
    comments=frame.body
    textrating=[]
    for comment in comments:
        textrating.append(textstat.flesch_kincaid_grade(comment))
    #textstat.flesch_reading_ease(test_data)
    #textstat.smog_index(test_data)
    #textstat.flesch_kincaid_grade(test_data)
    #textstat.coleman_liau_index(test_data)
    #textstat.automated_readability_index(test_data)
    #textstat.dale_chall_readability_score(test_data)
    #textstat.difficult_words(test_data)
    #textstat.linsear_write_formula(test_data)
    #textstat.gunning_fog(test_data)
    #textstat.text_standard(test_data)

    return textrating


In [28]:
#for length of comment
def wordLengthGenerator(frame):
    comments=frame.body
    lengths=[]
    for comment in comments:
        lengths.append(len(comment))
    return lengths



In [27]:
#returns the ratio of uppercase to lower case letters
#returns 0 if length is zero
def upperCaseGenerator(frame):
    comments=frame.body
    ratios=[]
    for comment in comments:
        count = 0
        for c in comment:
            if(c.isupper()):
                count = count + 1
        if(len(comment)>0):
            ratioval=float(count/len(comment))
        else:
            ratioval=0
        ratios.append(ratioval)
    return ratios



In [31]:
#counts number of questions in a comment
#by counting number of question marks
def numberOfQuestionsInAComment(frame):
    comments=frame.body
    questions=[]
    for comment in comments:
        count=0
        for c in comment:
            if(c=='?'):
                count=count+1
        questions.append(count)
    return questions


In [38]:
#naive method of determining whether or
#not the title is a question
def isTitleAQuestion(frame):
    START_WORDS = ["who", "what", "when", "where", "why", "how", "is", "can", "does", "do",
                  "could","should","would","which","whose","whom","are",]
    permalinks=frame.permalink
    isQuestion=[]
    for permalink in permalinks:
        splitty=permalink.split('/')
        title=splitty[5]
        wordsInTitle=title.split('_')
        firstWord=wordsInTitle[0]
        if(firstWord in START_WORDS):
            isQuestion.append(1)
        else:
            isQuestion.append(0)
    
    return isQuestion


[0]

In [ ]:
# subreddit name separation and extraction

In [17]:
model = Doc2Vec.load("final_d2v_model.model")

In [25]:
print(model.docvecs.most_similar(0))

C:\Users\Risha\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('2160395', 0.773759126663208), ('1433720', 0.7351023554801941), ('2640563', 0.7267253994941711), ('769285', 0.7115885019302368), ('1871186', 0.7106360197067261), ('826092', 0.6876705288887024), ('1854229', 0.6779251098632812), ('381699', 0.6749313473701477), ('1423085', 0.6657606363296509), ('1966050', 0.6602393984794617)]


In [17]:
# words = "king queen man".split()
# new_vec = model.infer_vector(words)
def similar(i, m):
    print("Original: ", frame['body'][i].replace("\r", " "))
    similar_i = m.docvecs.most_similar(i)
    for doc_i, sim in similar_i:
        print("Similarity: ", str(sim), "\nDoc ", str(doc_i), ":\n", frame['body'][int(doc_i)].replace("\r", " "))

In [78]:
print(frame['body'][3])

 Why’d ya have to bring race or weight into it? Yeesh kid. 


In [79]:
carriage = [(i, body) for i, body in enumerate(frame['body']) if '\r' in body]

In [90]:
print("dfgdfg\rsdffdsdfgsdfg")

sdffdsdfgsdfg


In [80]:
carriage[:10]

[(3,
  'What? \r It’s a niche rpg. Most rpg fans are adults. \r Why’d ya have to bring race or weight into it? Yeesh kid. '),
 (8,
  "I'm sorry that happened to you, I had a similar thing happen to me and it worked out okay. I went to a first aid class with the rest of my uni cohort and fainted after doing CPR (I was doing a fitness challenge at the time and probably hadn't eaten/drunken enough that day, so my own fault really). I was out for maybe 10 seconds at most, and when I came to my classmates were all super concerned. One guy gave me his sandwich, another went to buy me some Powerade and bananas. I went home, then the next time I went to class a couple of friends asked me if I was okay and that was it. Never brought up again, I wasn't the focus of attention at all. It was just a couple of friends checking up on me, as if I'd just had the flu or something. \r \r And I also kinda dealt with it through humour, which is kinda my go-to. For example, I had to do a speech later that y

In [85]:
tagged_data = []
for i, _d in carriage[:10]:
    tagged_data.append(TaggedDocument(words=word_tokenize(_d.lower()),tags=[str(i)]))

In [86]:
tagged_data[0]

TaggedDocument(words=['what', '?', 'it', '’', 's', 'a', 'niche', 'rpg', '.', 'most', 'rpg', 'fans', 'are', 'adults', '.', 'why', '’', 'd', 'ya', 'have', 'to', 'bring', 'race', 'or', 'weight', 'into', 'it', '?', 'yeesh', 'kid', '.'], tags=['3'])

In [74]:
print(similar_i)

[('2160395', 0.773759126663208), ('1433720', 0.7351023554801941), ('2640563', 0.7267253994941711), ('769285', 0.7115885019302368), ('1871186', 0.7106360197067261), ('826092', 0.6876705288887024), ('1854229', 0.6779251098632812), ('381699', 0.6749313473701477), ('1423085', 0.6657606363296509), ('1966050', 0.6602393984794617)]


In [18]:
similar(434, model)

Original:  Man I'm jelly you got 2, but after playing with a 55% I realise you don't even need a dupe on him the team is so op. I tried not to even super attack to get the transformation and even then it's pretty hard and only possible on last stage.
Similarity:  0.7767763137817383 
Doc  551337 :
 lol no problem. do u need the LR as lead or Teq VB? I got both
Similarity:  0.7724827527999878 
Doc  1802996 :
 Because it's SUPER good in this patch. In 90% of my games, all I get to do is farm. And in these types of games you just farm for the first 30min as jugg for example and get bfury-&gt;manta-&gt;blink and just steamroll over the opponents.    Currently the meta is either all in fight (vs lycan lineups etc) and then jugg sucks. Or its a tradeoff between cores with farm.
Similarity:  0.7687543630599976 
Doc  572361 :
 Yeah but I've only got that on Zelgius and then the one guy who's got it as a seal. I run into at least 4 Ninos an Arena Assault run anymore.    I guess I can make use of

C:\Users\Risha\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [107]:
frame['body'][421663].replace('\r', ' ')

'It was '

In [20]:
def similar_to(doc, m, df):
    words = word_tokenize(doc.lower())
    vec = m.infer_vector(words)
    print("Original: ", doc.replace("\r", " "))
    similar_i = m.docvecs.most_similar([vec])
    for doc_i, sim in similar_i:
        print("Similarity: ", str(sim), "\nDoc ", str(doc_i), ":\n", df['body'][int(doc_i)].replace("\r", " "))

In [143]:
words = word_tokenize("avatar sucks".lower())
vec = model.infer_vector(words, epochs=500)
#print(vec)
print("Original: ", "avatar sucks".replace("\r", " "))
similar_i = model.docvecs.most_similar([vec])
for doc_i, sim in similar_i:
    print("Similarity: ", str(sim), "\nDoc ", str(doc_i), ":\n", frame['body'][int(doc_i)].replace("\r", " "))

Original:  avatar sucks


C:\Users\Risha\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Similarity:  0.5717811584472656 
Doc  422507 :
 james johnson sucks
Similarity:  0.5379082560539246 
Doc  464461 :
 Sucks to be you[.](https://imgur.com/cXA7XxW)
Similarity:  0.5319690704345703 
Doc  2411277 :
 Hollywood sucks sucks sucks 
Similarity:  0.5304902195930481 
Doc  1915666 :
 This guy sucks ^
Similarity:  0.5294104218482971 
Doc  1774504 :
 Sucks to suck.
Similarity:  0.5059899091720581 
Doc  318465 :
 One that sucks sucks . 
Similarity:  0.504196286201477 
Doc  2108386 :
 This Black Mirror episode sucks
Similarity:  0.5040415525436401 
Doc  2873830 :
 Being childless and 60+ years old sucks. 
Similarity:  0.4917881190776825 
Doc  2580845 :
 Josh Allen sucks. 
Similarity:  0.49134188890457153 
Doc  2303977 :
 Buying and selling houses sucks. A lot.


In [21]:
similar_to("airbender", model, frame)

Original:  airbender
Similarity:  0.7820019125938416 
Doc  604163 :
 .
Similarity:  0.7577211260795593 
Doc  221724 :
 I
Similarity:  0.7492688298225403 
Doc  1853457 :
 Glaub eine Frau
Similarity:  0.7422033548355103 
Doc  1214998 :
 What a *darn* shame..  ***  ^^Darn ^^Counter: ^^56396
Similarity:  0.7377957105636597 
Doc  1931480 :
 150 Partnership 
Similarity:  0.7355886697769165 
Doc  1408156 :
 Ist ein Fehler. 
Similarity:  0.7283194065093994 
Doc  1049938 :
 .
Similarity:  0.7213022112846375 
Doc  353249 :
 .
Similarity:  0.7200831174850464 
Doc  2135047 :
 Ich liebe dich, Özil 
Similarity:  0.7181810140609741 
Doc  2220896 :
 I


C:\Users\Risha\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
